In [27]:
import numpy as np
import cv2
import os

classes    = ["cellphone", "flamingo", "Motorbikes"]
train_path = "CaltechTinySplit/train/"
test_path  = "CaltechTinySplit/test/"


Tanımlanan sınıflardaki folder path'ini bir metod ile ele alarak burada bulunan tüm resimleri opencv (cv2) kütüphanesi ile tek tek okuyarak np.array tipinde liste ile döndürmeliyiz


In [28]:
def load_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64)).flatten()  # 64x64 boyutuna küçültüp düzleştirme
        images.append(img)
    return np.array(images).T  # Boyut: 4096 x n

Tanımlanan metodu kullanacak ve tüm class verilerini (ortalama matrisi ve kovaryans matrisi) hesaplayacak bir döngü yazılarak bu değerlerin bir listeye eklenmesi sağlanmalıdır.

In [29]:
# Modelleri saklamak için liste
models = []

# Her sınıf için işlem
for class_folder in classes:
    train_images_path = train_path + class_folder
    print("train_images_path", train_images_path)
    train_data = load_images(train_images_path)
    # Her bir sınıf için hesaplanan metrikler models listesinde tutulur. [ortalama matrisi ve kovaryans matrisi]
    mu_i = np.mean(train_data, axis=1, keepdims=True)  # 4096x1
    sigma_i = np.cov(train_data)  # 4096x4096
    sigma_inv_i = np.linalg.pinv(sigma_i)
    models.append((mu_i, sigma_inv_i))

train_images_path CaltechTinySplit/train/cellphone
train_images_path CaltechTinySplit/train/flamingo
train_images_path CaltechTinySplit/train/Motorbikes


In [36]:
mu_i.shape

(4096, 1)

In [37]:
sigma_i.shape

(4096, 4096)

In [38]:
def mahalanobis_distance(x, mu, sigma_inv):
    diff = x - mu
    return diff.T @ sigma_inv @ diff

test verilerini de tıpkı train verilerinde yaptığımız gibi vektörel hale getirip, ayrıca label'ları ile bir listede tutulmalıdır.

In [ ]:
test_data = []
test_labels = []  # Gerçek sınıf etiketlerini saklamak için liste

for class_idx, class_folder in enumerate(classes):
    test_images_path = test_path + class_folder
    print("test_images_path", test_images_path)
    class_test_data = load_images(test_images_path)
    test_data.append(class_test_data)
    test_labels += [class_idx] * class_test_data.shape[1]

# ayrı listeler birleştirilmeli    
test_data = np.hstack(test_data)
test_labels = np.array(test_labels)


test_images_path CaltechTinySplit/test/cellphone
test_images_path CaltechTinySplit/test/flamingo
test_images_path CaltechTinySplit/test/Motorbikes


In [40]:
test_data.shape

(4096, 96)

Verilerin predict edilmesi için ilk olarak test data'da bulunan her veri için train verilerinden hesapladığımız ortalama ve kovaryans matrislerine göre mahalanobis uzaklığı hesaplanmalıdır. Bu uzaklıkların en küçüğüne göre test verisinin hangi sınıfa ait olduğu belirlenmelidir.

In [45]:
predictions = []
for i in range(test_data.shape[1]):  # Test verisindeki her örnek için
    x_test = test_data[:, i:i+1]  # 4096x1 formatında vektör al
    min_distance = float("inf")
    predicted_class = -1

    for class_idx, (mu, sigma_inv) in enumerate(models):
        distance = mahalanobis_distance(x_test, mu, sigma_inv)
        if distance < min_distance:
            min_distance = distance
            predicted_class = class_idx
    
    predictions.append(predicted_class)  # Sonuçları listeye ekle
    print(f"Test image {i} is classified as {classes[predicted_class]}, real class is {classes[test_labels[i]]}")

Test image 0 is classified as flamingo, real class is cellphone
Test image 1 is classified as cellphone, real class is cellphone
Test image 2 is classified as flamingo, real class is cellphone
Test image 3 is classified as Motorbikes, real class is cellphone
Test image 4 is classified as flamingo, real class is cellphone
Test image 5 is classified as Motorbikes, real class is cellphone
Test image 6 is classified as cellphone, real class is cellphone
Test image 7 is classified as cellphone, real class is flamingo
Test image 8 is classified as flamingo, real class is flamingo
Test image 9 is classified as flamingo, real class is flamingo
Test image 10 is classified as cellphone, real class is flamingo
Test image 11 is classified as Motorbikes, real class is flamingo
Test image 12 is classified as Motorbikes, real class is flamingo
Test image 13 is classified as flamingo, real class is flamingo
Test image 14 is classified as Motorbikes, real class is flamingo
Test image 15 is classified a

In [46]:
predictions = np.array(predictions)  # Boyut: (m,)


Mode Doğruluğunun hesaplanması

In [48]:
# Model doğruluğunu hesapla
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_labels, predictions)
print(f"Model Doğruluk Oranı: {accuracy * 100:.2f}%")


Model Doğruluk Oranı: 23.96%


Modelin ne kadar doğru tahmin ettiğini ölçmek için sklearn.metrics içerisinden accuracy_score fonksiyonu kullanıldı. 

Model %23.96 oranında doğru tahmin yaptığı görüldü, bu oranın arttırılması için : 
- Daha fazla veri ile modelin eğitilmesi, 
- Verilerin shuffle edilmesi,
- Daha fazla feature eklenmesi,
- Daha farklı bir modelin kullanılması

gibi yöntemler denenmelidir.

### Shuffle Kullanılarak Modelin eğitilmesi

Veriler shuffle edilerek baştan eğitilirse modelin daha iyi sonuçlar vermesi beklenir.

In [ ]:
from sklearn.utils import shuffle

In [50]:

# Modelleri saklamak için liste
models = []

# Her sınıf için eğitim verisini yükle
for class_idx, class_folder in enumerate(classes):
    train_images_path = train_path + class_folder
    print("Eğitim verisi klasörü:", train_images_path)

    train_data = load_images(train_images_path)

    if train_data.size == 0:
        print(f"Uyarı: {class_folder} için veri bulunamadı, atlanıyor.")
        continue  # Boş sınıf varsa işlemi atla

    # **Verileri Shuffle Et**
    train_data = shuffle(train_data.T, random_state=42).T  # Shuffle ve Transpose

    # Ortalama ve kovaryans hesaplama
    mu_i = np.mean(train_data, axis=1, keepdims=True)  # 4096x1
    sigma_i = np.cov(train_data, rowvar=True)  # 4096x4096

    # Kovaryans matrisinin tersini önceden hesapla
    sigma_inv_i = np.linalg.pinv(sigma_i)

    models.append((mu_i, sigma_inv_i))  # Sigma'nın tersini sakla


Eğitim verisi klasörü: CaltechTinySplit/train/cellphone
Eğitim verisi klasörü: CaltechTinySplit/train/flamingo
Eğitim verisi klasörü: CaltechTinySplit/train/Motorbikes


Train verilerinin de aynı şekilde yanlı olmaması için shuffle edilerek değerlendirilmesi gerekir. 

In [53]:
predictions = []
test_data = []
test_labels = []  # Gerçek sınıf etiketlerini saklamak için liste

# Test verisini yükleme
for class_idx, class_folder in enumerate(classes):
    test_images_path = test_path + class_folder
    print("Test verisi klasörü:", test_images_path)
    
    class_test_data = load_images(test_images_path)  # Her sınıfın test verisini yükle

    if class_test_data.size == 0:
        print(f"Uyarı: {class_folder} için test verisi yok, atlanıyor.")
        continue  # Eğer test verisi boşsa atla

    test_data.append(class_test_data)  # Test verilerini ekle
    test_labels.extend([class_idx] * class_test_data.shape[1])  # Gerçek etiketleri ekle

# Test verilerini tek bir NumPy dizisine çevir
test_data = np.hstack(test_data)  # Boyut: (4096, m)
test_labels = np.array(test_labels)  # Boyut: (m,)

# **Verileri Shuffle Et**
test_data, test_labels = shuffle(test_data.T, test_labels, random_state=42)  # Shuffle ve Transpose
test_data = test_data.T  # Tekrar (4096, m) formata çevir

# Test verisini tahmin etme
for i in range(test_data.shape[1]):  # Test verisindeki her örnek için
    x_test = test_data[:, i:i+1]  # 4096x1 formatında vektör al
    min_distance = float("inf")
    predicted_class = -1

    for class_idx, (mu, sigma_inv) in enumerate(models):
        distance = mahalanobis_distance(x_test, mu, sigma_inv)
        if distance < min_distance:
            min_distance = distance
            predicted_class = class_idx
    
    predictions.append(predicted_class)  # Sonuçları listeye ekle
    print(f"Test image {i} is classified as {classes[predicted_class]}, real class is {classes[test_labels[i]]}")

# NumPy formatına çevir
predictions = np.array(predictions)  # Boyut: (m,)




Test verisi klasörü: CaltechTinySplit/test/cellphone
Test verisi klasörü: CaltechTinySplit/test/flamingo
Test verisi klasörü: CaltechTinySplit/test/Motorbikes
Test image 0 is classified as cellphone, real class is Motorbikes
Test image 1 is classified as Motorbikes, real class is Motorbikes
Test image 2 is classified as Motorbikes, real class is Motorbikes
Test image 3 is classified as Motorbikes, real class is Motorbikes
Test image 4 is classified as cellphone, real class is Motorbikes
Test image 5 is classified as Motorbikes, real class is Motorbikes
Test image 6 is classified as cellphone, real class is Motorbikes
Test image 7 is classified as Motorbikes, real class is Motorbikes
Test image 8 is classified as cellphone, real class is cellphone
Test image 9 is classified as cellphone, real class is flamingo
Test image 10 is classified as flamingo, real class is Motorbikes
Test image 11 is classified as cellphone, real class is Motorbikes
Test image 12 is classified as Motorbikes, rea

In [54]:
# Model doğruluğunu hesapla
accuracy = accuracy_score(test_labels, predictions)
print(f"Model Doğruluk Oranı: {accuracy * 100:.2f}%")

Model Doğruluk Oranı: 57.29%


Görüldüğü gibi shuffle edilerek verilerden daha iyi öğrenilebildiği görülmüştür, resim örnek sayısının artırılması da modelin daha iyi öğrenmesine yardımcı olacaktır. 

Özet olarak train verilerinden hazırlanan ortalama ve kovaryans matrisleri ile test verileri karşılaştırılarak en yakın olan sınıfın tahmin edilmesi sağlanmıştır. Bu yakınlık değeri mahalanobis uzaklığı ile hesaplanmıştır. 